In [208]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
import torch
import json
import uuid
from queue import Queue
import torch.nn as nn
import torch.optim as optim
import opendatasets as od
od.download("https://www.kaggle.com/datasets/mkechinov/ecommerce-events-history-in-cosmetics-shop/code?dataset")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Skipping, found downloaded files in "./ecommerce-events-history-in-cosmetics-shop" (use force=True to force download)


In [209]:
# Read each of the csv files
df_1910 = pd.read_csv('Downloads/archive/2019-Oct.csv')
df_1911 = pd.read_csv('Downloads/archive/2019-Nov.csv')
df_1912 = pd.read_csv('Downloads/archive/2019-Dec.csv')
df_2001 = pd.read_csv('Downloads/archive/2020-Jan.csv')
df_2002 = pd.read_csv('Downloads/archive/2020-Feb.csv')
data = pd.concat([df_1910, df_1911, df_1912, df_2001, df_2002], ignore_index= True)
data.head(50)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,cart,5773203,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
1,2019-10-01 00:00:03 UTC,cart,5773353,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
2,2019-10-01 00:00:07 UTC,cart,5881589,2151191071051219817,NaN,lovely,13.48,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9
3,2019-10-01 00:00:07 UTC,cart,5723490,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
4,2019-10-01 00:00:15 UTC,cart,5881449,1487580013522845895,NaN,lovely,0.56,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9
5,2019-10-01 00:00:16 UTC,cart,5857269,1487580005134238553,NaN,runail,2.62,430174032,73dea1e7-664e-43f4-8b30-d32b9d5af04f
6,2019-10-01 00:00:19 UTC,cart,5739055,1487580008246412266,NaN,kapous,4.75,377667011,81326ac6-daa4-4f0a-b488-fd0956a78733
7,2019-10-01 00:00:24 UTC,cart,5825598,1487580009445982239,NaN,NaN,0.56,467916806,2f5b5546-b8cb-9ee7-7ecd-84276f8ef486
8,2019-10-01 00:00:25 UTC,cart,5698989,1487580006317032337,NaN,NaN,1.27,385985999,d30965e8-1101-44ab-b45d-cc1bb9fae694
9,2019-10-01 00:00:26 UTC,view,5875317,2029082628195353599,NaN,NaN,1.59,474232307,445f2b74-5e4c-427e-b7fa-6e0a28b156fe


In [210]:
class FakeKafkaProducer:
    def __init__(self):
        self.messages = []
        
    def produce(self, topic, message):
        self.messages.append((topic, message))

In [211]:
class FakeKafkaConsumer:
    def __init__(self, topic):
        self.topic = topic
        self.queue = Queue()
        
    def consume(self):
        # Get a message from the queue
        message = self.queue.get()
        # Return the message value
        return message['value']
    
    def poll(self):
        # Check if there are any messages in the queue
        while not self.queue.empty():
            # Get a message from the queue
            message = self.queue.get()
            # Yield the message value
            yield message['value']
    
    def add_message(self, message):
        # Add the message to the queue
        self.queue.put(message)

In [212]:
class FakeKafkaBroker:
    def __init__(self):
        self.topics = {}

    def create_topic(self, topic):
        if topic not in self.topics:
            self.topics[topic] = []

    def produce_message(self, topic, message):
        if topic not in self.topics:
            self.create_topic(topic)
        self.topics[topic].append(message)

    def consume_messages(self, topic):
        if topic not in self.topics:
            self.create_topic(topic)
        messages = self.topics[topic]
        self.topics[topic] = []
        return messages

In [213]:
def recommend(user_id, cart_items, interactions_df, item_metadata_df):
    # Filter the interactions data to include only interactions for the chosen user
    user_data = interactions_df[interactions_df['user_id'] == user_id]

    # Get a list of items the user has interacted with
    interacted_items = set(user_data['product_id'].tolist())

    # Filter the item metadata to include only items the user has not interacted with
    uninteracted_items = item_metadata_df[~item_metadata_df['product_id'].isin(interacted_items)]

    # Filter the user's cart to include only items that were never purchased
    unpurchased_items = cart_items[~cart_items['product_id'].isin(interacted_items)]

    # If there are unpurchased items in the cart, recommend the first one
    if len(unpurchased_items) > 0:
        recommended_item = unpurchased_items.iloc[0]['product_id']
    else:
        # Otherwise, recommend the most popular uninteracted item
        item_popularity = user_data.groupby('product_id')['event_type'].count().reset_index()
        item_popularity = item_popularity.merge(uninteracted_items, on='product_id', how='left')
        item_popularity = item_popularity.sort_values(by=['event_type', 'product_name'], ascending=[False, True])
        recommended_item = item_popularity.iloc[0]['product_id']

    return recommended_item

In [214]:
# Create a fake Kafka producer
producer = FakeKafkaProducer()

# Choose a specific user ID to analyze
user_id = 474232307

# Filter the data to include only interactions for the chosen user
user_data = data[data['user_id'] == user_id]

# Define the number of items to recommend
num_items = 10

# Define the number of previous items to consider
N = 10

# Define a list to store all the interactions for the specified user
user_history = []

# Define a list to store the ground truth data
ground_truth = []

# Define a dictionary to store the last N items for each user
last_n_items = {}

# Define a list to store the recommended items
recommended_items = []

# Loop over all user sessions for the specified user
for session in user_data['user_session'].unique():

    # Get the recommended item based on the user's previous interactions
    if session in last_n_items:
        recommended_item = last_n_items[session][-1]
    else:
        recommended_item = None

    # Get the current session interactions for the current user
    session_interactions = user_data[user_data['user_session'] == session].sort_values(by='event_time', ascending=False)['product_id'].tolist()

    # Update the user history with the current session interactions
    user_history.extend(session_interactions)

    # Keep track of only the last N interactions for the user
    if len(user_history) >= N:
        user_history = user_history[-N:]

    # Get the recommended item based on the user's previous interactions if no previous session exists
    if recommended_item is None and len(user_history) > 0:
        recommended_item = user_history[-1]

    # If the recommended item is not in the user's cart, recommend the first item in the cart that was never purchased
    if recommended_item is not None and recommended_item not in user_data[user_data['event_type'] == 'cart']['product_id'].tolist():
        cart_items = user_data[user_data['event_type'] == 'cart']['product_id'].tolist()
        purchased_items = user_data[user_data['event_type'] == 'purchase']['product_id'].tolist()
        for item in cart_items:
            if item not in purchased_items:
                recommended_item = item
                break

    # Append the recommended item to the list of recommended items
    recommended_items.append(recommended_item)

    # Update the list of last N items for the user session
    if session in last_n_items:
        last_n_items[session].extend(session_interactions)
        if len(last_n_items[session]) > N:
            last_n_items[session] = last_n_items[session][-N:]
    else:
        last_n_items[session] = session_interactions[-N:]

    # Append the ground truth item to the list
    ground_truth.append(user_data[user_data['user_session'] == session]['product_id'].iloc[0])

    # Simulate a new session for the user ID
    new_session_id = str(uuid.uuid4())

    # Update the last N items for the new session to be the last N items in the user history
    last_n_items[new_session_id] = user_history[-N:]

    # Send a new message to the Kafka topic for the new session
    new_message = {'session_id': new_session_id, 'user_id': user_id}
    producer.produce('user-events', new_message)

    # Get the recommended item for the new session based on the user's previous interactions
    if recommended_item is None and len(user_history) > 0:
        recommended_item = user_history[-1]

    # If the recommended item is not in the user's cart, recommend the first item in the cart that was never purchased
    if recommended_item is not None and recommended_item not in user_data[user_data['event_type'] == 'cart']['product_id'].tolist():
        cart_items = user_data[user_data['event_type'] == 'cart']['product_id'].tolist()
        purchased_items = user_data[user_data['event_type'] == 'purchase']['product_id'].tolist()
        for item in cart_items:
            if item not in purchased_items:
                recommended_item = item
                break

# Append the recommended item to the list of recommended items
recommended_items.append(recommended_item)

# Update the last N items for the new session to include the recommended item
last_n_items[new_session_id].append(recommended_item)

# Send a new message to the Kafka topic for the new session
new_message = {'session_id': new_session_id, 'user_id': user_id, 'recommended_item': recommended_item}
producer.produce('user-events', new_message)

    # Calculate the precision and recall
num_correct = len(set(ground_truth).intersection(set(recommended_items)))
precision = num_correct / num_items
recall = num_correct / len(ground_truth)

# Print the previous session for the current user
if session in last_n_items and len(last_n_items[session]) > 1:
    prev_session_items = last_n_items[session][:-1]
    print(f"Previous session items for user {user_id} and session {session}: {prev_session_items}")

# Print the new session ID and recommended item
print(f"New session ID: {new_session_id}")
print(f"Recommended item: {recommended_item}")

# Print the last N items for the new session
if new_session_id in last_n_items:
    print(f"Last {N} items for new session: {last_n_items[new_session_id]}")
else:
    print(f"No previous interactions for new session {new_session_id}")


print(f"Precision: {precision}")
print(f"Recall: {recall}")



New session ID: f544bc2c-8c2a-4ad0-a139-615fed3e360b
Recommended item: 5707757
Last 10 items for new session: [5809344, 5610125, 5877095, 5748921, 5770325, 5707958, 5896421, 5877528, 5894927, 5896169, 5707757]
Precision: 0.6
Recall: 0.375
User History: [5809344, 5610125, 5877095, 5748921, 5770325, 5707958, 5896421, 5877528, 5894927, 5896169]


In [215]:
user_id = '474232307'

# filter the DataFrame to get all the rows where the user ID matches
user_purchases = data[(data['user_id'] == user_id) & (data['event_type'] == 'purchase')]

# extract the list of items purchased by the user
user_items = user_purchases['product_id'].tolist()

# print the list of items
print(user_purchases)

Empty DataFrame
Columns: [event_time, event_type, product_id, category_id, category_code, brand, price, user_id, user_session]
Index: []


In [216]:
purchase_data = data[data['event_type'] == 'purchase']
print(purchase_data.head(10))

                  event_time event_type  product_id          category_id  \
202  2019-10-01 00:06:35 UTC   purchase     5619862  1487580006895846315   
204  2019-10-01 00:06:35 UTC   purchase     5619866  1487580006895846315   
205  2019-10-01 00:06:35 UTC   purchase     5696184  1487580005134238553   
206  2019-10-01 00:06:35 UTC   purchase     5706784  1487580005092295511   
207  2019-10-01 00:06:35 UTC   purchase     5723503  1487580005134238553   
208  2019-10-01 00:06:35 UTC   purchase     5773207  1487580005134238553   
209  2019-10-01 00:06:35 UTC   purchase     5814822  1487580005897601916   
210  2019-10-01 00:06:35 UTC   purchase     5854324  1487580005134238553   
211  2019-10-01 00:06:35 UTC   purchase     5865795  1487580008909111303   
212  2019-10-01 00:06:35 UTC   purchase     5875317  2029082628195353599   

    category_code   brand  price    user_id  \
202           NaN  runail   5.32  474232307   
204           NaN  runail   2.86  474232307   
205           NaN  run